# Проблема - местами 6 идет в ID сместо реального ID!!!

In [32]:
k=6
for number in range(1):
    k=round(k+1,2)
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import r2_score

    test_data = pd.read_csv('test.csv')
    train_data = pd.read_csv('train.csv')

    # Функции для очистки и подготовки данных
    mean_year = np.round(train_data.loc[train_data['HouseYear'] <= 2020, 'HouseYear'].mean())
    mean_healthcare = np.round(train_data["Healthcare_1"].mean())
    mean_square_for_max = train_data.loc[(train_data['Rooms'] <= train_data.loc[(train_data['Square'] > 300), 'Rooms'].mean()), 'Square'].mean()
    mean_square_for_big_ls = train_data.loc[train_data['LifeSquare'] > 250, 'Square'].mean()
    mean_life_squae_for_max = train_data.loc[train_data['Square'] >= mean_square_for_big_ls, 'LifeSquare'].mean()
    
    mean_square_Kitchen=train_data.KitchenSquare.mean()
    
    def clean_year(df, mean_year):
        df.loc[df['HouseYear'] > 2020, 'HouseYear'] = mean_year

    def clean_life_square(df, koef_S_LS):
        df.loc[(df['LifeSquare'] < 10) | (df['LifeSquare'].isnull()), 'LifeSquare'] = df['Square']*0.85
        df.loc[df['LifeSquare'] > 250, 'LifeSquare'] = mean_life_squae_for_max

    def clean_square(df, mean_square_for_max):
        df.loc[(df['Square'] > 300), 'Square'] = mean_square_for_max

    def clean_healthcare_1(df, mean_healthcare):
        df.loc[df['Healthcare_1'].isnull(), 'Healthcare_1'] = mean_healthcare

    def clean_rooms(df):
        df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] < 30), 'Rooms'] = 1
        df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 30) & (df['LifeSquare'] < 45), 'Rooms'] = 2
        df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 45) & (df['LifeSquare'] < 60), 'Rooms'] = 3
        df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 60) & (df['LifeSquare'] < 75), 'Rooms'] = 4
        df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 70), 'Rooms'] = 6
        df.loc[(df['Rooms'] > 10), 'Rooms'] = 2
    
    def KitchenSquare(df, mean_square_for_max):
        df.loc[(df['KitchenSquare'] < 4) | (df['KitchenSquare'].isnull()), 'KitchenSquare'] = 6
        df.loc[(df['KitchenSquare'] > 24)] = 6 # тут битый ID, который дает большое примущество! 
                                               # Исправить: , 'KitchenSquare'] = 6

    def prepare_data(df, mean_year=mean_year, mean_healthcare=mean_healthcare, mean_square_for_max=mean_square_for_max, mean_life_squae_for_max=mean_life_squae_for_max):
        clean_year(df, mean_year)
        clean_life_square(df, mean_life_squae_for_max)
        clean_healthcare_1(df, mean_healthcare)
        clean_rooms(df)
        clean_square(df, mean_square_for_max)
        KitchenSquare(df, mean_square_for_max)
        

    prepare_data(train_data)
    prepare_data(test_data)

    X = pd.get_dummies(train_data)
    X.drop("Price", axis=1, inplace=True)
    X.drop("Id", axis=1, inplace=True)
    y = train_data.Price

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)
    # переобучение и оценка модели
    from sklearn.ensemble import GradientBoostingRegressor
    final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
                                           )
    # min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

    final_model.fit(X_train, y_train)

    y_pred_gbr = final_model.predict(X_valid)
    y_pred_train_gbr = final_model.predict(X_train)

    print('r2: ', r2_score(y_valid, y_pred_gbr),', k: ',k)

r2:  0.7736291080503545 , k:  7


In [30]:
train_data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8345.857800,50.206900,1.908500,55.823954,40.135559,7.340100,8.515600,12.574600,1976.263400,0.144144,24.623100,5331.321900,8.05290,1137.649300,1.340400,4.243300,213046.940530
std,4877.457592,43.628945,0.863135,19.015934,20.290707,2.182638,5.227704,6.755531,130.774586,0.402749,17.535393,4011.958434,23.82817,738.952922,1.520315,4.801418,93456.505617
min,0.000000,0.000000,1.000000,1.136859,0.966330,4.000000,1.000000,0.000000,6.000000,0.000000,0.000000,6.000000,0.00000,0.000000,0.000000,0.000000,6.000000
25%,4119.750000,19.000000,1.000000,41.678137,26.961740,6.000000,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.000000,0.00000,810.000000,0.000000,1.000000,153071.420932
50%,8366.500000,34.500000,2.000000,52.323724,36.015564,6.000000,7.000000,12.000000,1977.000000,0.075779,25.000000,5285.000000,2.00000,1143.000000,1.000000,3.000000,191870.239695
75%,12572.250000,75.000000,2.000000,65.823051,48.755515,9.000000,12.000000,17.000000,2001.000000,0.195781,36.000000,7227.000000,5.00000,1143.000000,2.000000,6.000000,248640.135030
max,16798.000000,209.000000,10.000000,212.932361,427.584084,23.000000,42.000000,117.000000,2020.000000,6.000000,74.000000,19083.000000,141.00000,4849.000000,6.000000,23.000000,633233.466570


In [33]:
# Предсказываем цены для тестовых данных и выгружаем в файл
X_test = pd.get_dummies(test_data)
X_test.drop("Id", axis=1, inplace=True)
test_data["Price"] = final_model.predict(X_test)

In [34]:
# экспорт в файл
test_data.loc[:, ['Id', 'Price']].to_csv('best_gbr_06.csv', index=False)

0.7621056876187297 - test_size=0.16 - n_estimators=200, max_depth=5, random_state=42 (0.75339)

r2:  0.770945819723227 , k:  3-24 KS

r2:  0.7736291080503545 , k:  4-24 KS - с битым ID (0.76013)


In [25]:
test_data.Id.value_counts()

6        17
1066      1
701       1
10035     1
2740      1
         ..
5440      1
7489      1
1346      1
1350      1
16384     1
Name: Id, Length: 4984, dtype: int64

In [8]:
train_data.Id.value_counts()

6        44
2047      1
11543     1
7457      1
5408      1
         ..
2660      1
12899     1
8801      1
10848     1
0         1
Name: Id, Length: 9957, dtype: int64

In [16]:
# k=1000
# for number in range(200):
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)
#     # переобучение и оценка модели
#     from sklearn.ensemble import GradientBoostingRegressor
#     final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
#                                            )
#     # min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

#     final_model.fit(X_train, y_train)

#     y_pred_gbr = final_model.predict(X_valid)
#     y_pred_train_gbr = final_model.predict(X_train)

#     print('r2: ', r2_score(y_valid, y_pred),', n_estimators: ',k)

In [71]:
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
import seaborn as sns
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 9)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 9)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 9)))
    
    plt.figure(figsize=(10,10)) 
    
    sns.scatterplot(x=pred_values, y=true_values)
    
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.show()

In [76]:
# y_train_preds = final_model.predict(X_train)
# evaluate_preds(y_train, y_train_preds)